In [1]:
import numpy as np 
import h5py 
import sys,os
os.chdir('../../')
sys.path.append('./src/data')
sys.path.append('./src/events')
sys.path.append('./src/probability')
import pandas as pd 
from importergen2 import gen2_MC
from processergen2 import get_true_gen2,generate_probabilitiesgen2, get_probabilitiesgen2
from functions import ic_params_nsi

In [2]:
Ereco = np.array([5.623413,  7.498942, 10. , 13.335215, 17.782795, 23.713737, 31.622776, 42.16965 , 56.23413])
zreco = np.array([-1., -0.75, -0.5 , -0.25,  0., 0.25, 0.5, 0.75, 1.])

In [3]:
df = gen2_MC(track=True, cascade=False)

In [4]:
df

,reco_coszen,reco_energy,true_coszen,true_energy,rate_weight,pid,pdg
0,-0.658786,4.97,-0.580387,3.46,0.0,0.0,12.0
1,0.120503,13.20,0.020795,12.30,0.0,0.0,12.0
2,0.070737,14.60,-0.069148,12.30,0.0,0.0,12.0
3,-0.079121,6.37,-0.069148,4.15,0.0,0.0,12.0
4,-0.478727,2.88,-0.563985,1.89,0.0,0.0,12.0
...,...,...,...,...,...,...,...
271778,0.659232,20.10,-0.323290,47.60,0.0,0.0,-16.0
271779,0.305817,31.50,-0.049184,51.00,0.0,1.0,-16.0
271780,-0.275590,37.50,0.020795,124.00,0.0,0.0,-16.0
271781,0.647117,12.70,0.948287,16.70,0.0,0.0,-16.0


In [4]:
df['Ebin'] = pd.cut(df.reco_energy, Ereco, labels=False)
df['zbin'] = pd.cut(df.reco_coszen, zreco, labels=False)
df1 = (df
        .dropna()
        .drop('current_type',axis=1)
        .groupby(['pid','pdg','Ebin','zbin'])
        .sum()
        .reset_index())

In [5]:
df2 = pd.read_csv(f'./src/data/files/gen2/neutrino_mc.csv', dtype=np.float64)
df2['reco_coszen'] = np.cos(df2['reco_zenith'])
df2['true_coszen'] = np.cos(df2['true_zenith'])

In [6]:
df2

,pdg,pid,true_energy,true_zenith,true_azimuth,reco_energy,reco_zenith,reco_azimuth,interaction_type,current_type,weight,xsec,dxsec,x,y,W,Q2,reco_coszen,true_coszen
0,12.0,0.0,3.46,2.190,1.07,4.97,2.290,1.45,1.0,1.0,8.980000e-09,0.782,5.68,0.0879,0.1130,1.250,0.0648,-0.658786,-0.580387
1,12.0,0.0,12.30,1.550,5.39,13.20,1.450,5.86,2.0,1.0,2.180000e-07,70.200,116.00,0.4470,0.5520,2.820,5.7100,0.120503,0.020795
2,12.0,0.0,12.30,1.640,1.13,14.60,1.500,1.23,2.0,1.0,2.170000e-07,69.900,138.00,0.1020,0.7410,4.030,1.7400,0.070737,-0.069148
3,12.0,0.0,4.15,1.640,5.62,6.37,1.650,5.62,1.0,1.0,1.290000e-08,6.260,44.50,0.3510,0.1140,1.210,0.3130,-0.079121,-0.069148
4,12.0,0.0,1.89,2.170,3.33,2.88,2.070,3.16,0.0,1.0,1.450000e-09,7.600,1.11,2.0600,0.0374,0.861,0.2730,-0.478727,-0.563985
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271778,-16.0,0.0,47.60,1.900,2.99,20.10,0.851,3.01,2.0,0.0,9.600000e-06,1.080,6.61,0.1150,0.1130,3.130,1.1600,0.659232,-0.323290
271779,-16.0,1.0,51.00,1.620,1.91,31.50,1.260,1.70,2.0,0.0,1.150000e-05,4.790,22.80,0.0833,0.6940,7.870,5.5500,0.305817,-0.049184
271780,-16.0,0.0,124.00,1.550,2.61,37.50,1.850,2.69,2.0,0.0,1.170000e-04,50.000,86.30,0.3360,0.2990,6.870,23.4000,-0.275590,0.020795
271781,-16.0,0.0,16.70,0.323,2.23,12.70,0.867,2.60,2.0,0.0,6.590000e-07,2.920,1.49,0.2230,0.7280,4.310,5.0900,0.647117,0.948287


In [7]:
df2 = get_true_gen2(flavor='e',anti=False,pid=1,E_bin=0,z_bin=0,df=df2)
E,z = df2.true_energy.values, df2.true_coszen.values

In [8]:
#P = generate_probabilitiesgen2('e','m',E,z,0,0,ic_params_nsi,False,1,3,True)
P = get_probabilitiesgen2('e','m',0,0,ic_params_nsi,False,1,3)

In [11]:
P = P.reshape(-1,)

In [13]:
np.sum(df2['weight']*P)

4.698356189153638e-07